### python -m pip install pythonnet

In [481]:
from pathlib import Path
from enum import Enum
import json

In [482]:
data_folder = Path('./test_data/')
material_data = data_folder / 'material_data.json'
column_data = data_folder / 'column_data.json'
cross_section_data = data_folder / 'cross_section_data.json'
analytical_levels = data_folder / 'analytical_levels.json'

beam_cross_section_data = data_folder / 'beam_cross_section_data.json'
beam_data = data_folder / 'beam_data.json'
floor_data = data_folder / 'floor_data.json'
floor_sections_data = data_folder / 'floor_sections.json'
wall_data = data_folder / 'wall_data.json'
wall_sections_data = data_folder / 'wall_sections.json'

#start ETABS application.
myETABSObject.ApplicationStart()

#close the program.
ret = myETABSObject.ApplicationExit(False)


In [483]:
import clr
clr.AddReference("System.Runtime.InteropServices")
from System.Runtime.InteropServices import Marshal

#set the following path to the installed ETABS program directory
clr.AddReference(R'C:\Program Files\Computers and Structures\ETABS 22\ETABSv1.dll')
import ETABSv1 as etabs

#create API helper object
helper = etabs.cHelper(etabs.Helper())

try:
    myETABSObject = etabs.cOAPI(helper.GetObject("CSI.ETABS.API.ETABSObject"))
except:
    print("No running instance of the program found or failed to attach.")

#create SapModel object
SapModel = etabs.cSapModel(myETABSObject.SapModel)

N_mm_C = etabs.eUnits.N_mm_C
kN_m_C = etabs.eUnits.kN_m_C

display(myETABSObject)
display(SapModel)



# create stories

In [484]:
with open(analytical_levels, 'r') as file:
    levels = json.load(file)
level_dict = {k:v for k,v in levels}
sorted_level_dict = {k: v for k, v in sorted(level_dict.items(), key=lambda item: item[1])}
story_names = list(sorted_level_dict.keys())[1:]
story_heights = []
base_level = 0
for i,level in enumerate(sorted_level_dict.values()):
    if i == 0:
        base_level = level
        previous_level = level
        continue
    story_height = level - previous_level
    previous_level = level
    story_heights.append(story_height)

story_names
story_heights

def create_stories(SapModel,base_level:float,story_names:list[str],story_heights:list[float],unit = N_mm_C) -> tuple[int,list[str],list[float]]:
    ret = SapModel.SetPresentUnits(unit)
    stories_count = len(story_names)
    IsMasterStory = [False] * stories_count
    SimilarToStory = [''] * stories_count
    SpliceAbove = [False] * stories_count
    SpliceHeight = [0.0] * stories_count
    color = [0] * stories_count
    [ret, story_names, story_heights, IsMasterStory, SimilarToStory, SpliceAbove, SpliceHeight, color] = SapModel.Story.SetStories_2(
        base_level,
        stories_count,
        story_names,
        story_heights,
        IsMasterStory,
        SimilarToStory,
        SpliceAbove,
        SpliceHeight,
        color
    )
    return (ret, list(story_names), list(story_heights))

ret,_,_ = create_stories(SapModel,base_level,story_names,story_heights)


# create materials

In [485]:
with open(material_data, 'r') as file:
    materials = json.load(file)
materials

etabs_materials = {
    'concrete' : etabs.eMatType.Concrete,
    'steel': etabs.eMatType.Steel,
    'no_design' : etabs.eMatType.NoDesign
}


def get_etabs_concrete_material(name:str):
    return {
        'Name':name,
        'Fc':float(name[2:]),
        'IsLightweight':False,
        'FcsFactor':0.0,
        'SSType':2,
        'SSHysType':4,
        'StrainAtFc':0.0022,
        'StrainUltimate':0.005,
        'FinalSlope':-0.1,
        'FrictionAngle': 0.0,
        'DilatationalAngle' : 0.0,
        'Temp' : 0.0
    }


def get_etabs_steel_material(name:str):
    if name.upper() == 'S235':
        fy,fu,efy,efu = 235.0, 360.0, 258.5, 396.0
    elif name.upper() == 'S275':
        fy,fu,efy,efu = 275.0, 430.0, 302.5, 473.0
    elif name.upper() == 'S355':
        fy,fu,efy,efu = 355.0, 510.0, 390.5, 561.0
    elif name.upper() == 'S450':
        fy,fu,efy,efu = 440.0, 550.0, 484.0, 605.0
    elif name.upper() == 'A992FY50':
        fy,fu,efy,efu = 344.738, 448.159, 379.212, 492.975
    elif name.upper() == 'A572GR50':
        fy,fu,efy,efu = 344.738, 448.159, 379.212, 492.975
    elif name.upper() == 'A913GR50':
        fy,fu,efy,efu = 344.738, 413.685, 379.212, 455.054
    else:
        fy,fu,efy,efu = 248.211, 399.896, 372.317, 439.885

    return {
    'Name' : name,
    'Fy' : fy,
    'Fu' : fu,
    'EFy' : efy,
    'EFu' : efu,
    'SSType' : 1,
    'SSHysType' : 1,
    'StrainAtHardening' : 0.015,
    'StrainAtMaxStress' : 0.11,
    'StrainAtRupture' : 0.17,
    'FinalSlope' : -0.1,
    'Temp' : 0
    }



def add_material(SapModel, name:str,unit = N_mm_C):
    
    if name.upper().startswith('FC'):
        material_type = etabs_materials['concrete']
        fc = float(name[2:])
        E = 4700 * (fc)**0.5
        U = 0.2
        A = 0.0000099
        design_material = get_etabs_concrete_material(name)
    elif name.upper().startswith('S'):
        material_type = etabs_materials['steel']
        E = 199947.98
        U = 0.3
        A = 0.0000117
        design_material = get_etabs_steel_material(name)
    else:
        material_type = etabs_materials['no_design']
        E = 199947.98
        U = 0.3
        A = 0.0000117


    ret = SapModel.SetPresentUnits(unit)
    ret = SapModel.PropMaterial.SetMaterial(name, material_type)

    ret = SapModel.PropMaterial.SetMPIsotropic(
        name,
        E,
        U,
        A,
        Temp = 0
    )

    if material_type == etabs_materials['concrete']:
        ret = SapModel.PropMaterial.SetOConcrete_1(**design_material)
    elif material_type == etabs_materials['steel']:
        ret = SapModel.PropMaterial.SetOSteel_1(**design_material)


    ret = SapModel.SetPresentUnits(kN_m_C)
    weight = 25 if material_type == etabs_materials['concrete'] else 78.5
    ret = SapModel.PropMaterial.SetWeightAndMass(name, 1, weight)

    ret = SapModel.SetPresentUnits(unit)


_ = [add_material(SapModel,x) for x in materials]


# create cross sections

In [486]:
with open (cross_section_data, 'r') as file:
    sections = json.load(file)


In [487]:
def add_rebar_circular(section_name:str, rebars:str):
    rebar_count, rebar_size = rebars.upper().split('T')
    SetRebarColumn = {
        'Name': section_name,
        'MatPropLong':'A615Gr60',
        'MatPropConfine':'A615Gr60',
        'Pattern': 2,
        'ConfineType': 1,
        'Cover':40,
        'NumberCBars': int(rebar_count),
        'NumberR3Bars': 0,
        'NumberR2Bars': 0,
        'RebarSize': str(rebar_size),
        'TieSize':'10',
        'TieSpacingLongit':150,
        'Number2DirTieBars':3,
        'Number3DirTieBars':3,
        'ToBeDesigned': False
    }
    return SetRebarColumn

def add_rebar_rectangular(section_name:str, rebars:str,width_T2:float,depth_T3:float):
    rebar_count, rebar_size = rebars.upper().split('T')
    half_count_etabs = (int(rebar_count)-4) * 0.5 # without 4 bars at corner
    width_ratio = width_T2 / (width_T2 + depth_T3)
    NumberR3Bars = int(width_ratio * half_count_etabs)
    NumberR2Bars = max(int(half_count_etabs - NumberR3Bars),0) + 2
    NumberR3Bars += 2
    SetRebarColumn = {
        'Name': section_name,
        'MatPropLong':'A615Gr60',
        'MatPropConfine':'A615Gr60',
        'Pattern': 1,
        'ConfineType': 0,
        'Cover':40,
        'NumberCBars': 0,
        'NumberR3Bars': NumberR3Bars,
        'NumberR2Bars': NumberR2Bars,
        'RebarSize': str(rebar_size),
        'TieSize':'10',
        'TieSpacingLongit':150,
        'Number2DirTieBars':2,
        'Number3DirTieBars':4,
        'ToBeDesigned': False
    }
    return SetRebarColumn



In [488]:
# do not use this function , use the updated below
def get_etabs_column_concrete_section(section_data):
    size = str(section_data[1]['name']).split('_')[2]
    if str(section_data[1]['material_grade']).upper().startswith('FC'):
        if size.upper().startswith('D'):
            size = size[1:]
        name = f"{section_data[0]}-C {size}-{section_data[1]['rebar']}"
    else:
        name = f"{section_data[0]}-C {size}"
    return name

def get_etabs_column_concrete_section(section_data):
    size = str(section_data[1]['name']).split('_')[2]
    if str(section_data[1]['material_grade']).upper().startswith('FC'):
        if size.upper().startswith('D'):
            size = size[1:]
        name = f"C {size}-{section_data[1]['rebar']}-{section_data[1]['material_grade']}"
    else:
        name = f"C {size}-{section_data[1]['material_grade']}"
    return name


def add_sections(SapModel, section_data, unit = N_mm_C):
    ret = SapModel.SetPresentUnits(unit)
    name = get_etabs_column_concrete_section(section_data)
    material:str = section_data[1]['material_grade']
    rebars = section_data[1]['rebar']
    if section_data[1]['h']:
        width_T2 = float(section_data[1]['b'])
        depth_T3 = float(section_data[1]['h'])
        #define rectangular frame section property
        ret = SapModel.PropFrame.SetRectangle(name, material, depth_T3, width_T2)
        if rebars:
            ret = SapModel.PropFrame.SetRebarColumn(**add_rebar_rectangular(name,rebars,width_T2, depth_T3))
    else:
        depth_T3 = float(section_data[1]['b'])
        ret = SapModel.PropFrame.SetCircle(name, material, depth_T3)
        if rebars:
            ret = SapModel.PropFrame.SetRebarColumn(**add_rebar_circular(name,rebars))
    if material.upper().startswith('FC'):
        #define frame section property modifiers
        ModValue = [1.0, 1.0, 1.0, 1.0, 0.7, 0.7, 1.0, 1.0]
        ret = SapModel.PropFrame.SetModifiers(name, ModValue)
    return name

etabs_section_names = [add_sections(SapModel,x) for x in sections]

etabs_section_names


['C 500x900-16T20-FC40',
 'C 900-24T20-FC30',
 'C 400x800-12T16-FC30',
 'C 500x900-18T20-FC40',
 'C 800x800-20T20-FC30',
 'C 450x450-8T16-FC30',
 'C 800x800-20T20-FC40',
 'C HE360A-S355',
 'C SHS200x200x10-S275',
 'C 500x900-None-FC40',
 'C 500x900-None-FC30']

# creating columns in etabs

In [489]:
with open (column_data, 'r') as file:
    columns = json.load(file)
columns = sorted(columns, key= lambda x : x[1]['z_base'])

# correct coordinates to follow column below
def update_columns_coordinates(columns):
    updated_columns = []
    for column in columns:
        id = column[0]
        column_below_id = column[1]['column_below']
        if column_below_id:
            updated_column = [id, {k:v for k,v in column[1].items()}]
            for column_below in columns:
                if column_below_id == column_below[0]:
                    updated_column[1]['x'] = column_below[1]['x']
                    updated_column[1]['y'] = column_below[1]['y']
                    updated_columns.append(updated_column)
                    break
        else:
            updated_columns.append(column)
    return updated_columns

sorted_columns = update_columns_coordinates(columns)
def get_column_coordinate(column):
    name = column[0]

    x = column[1]['x']
    y = column[1]['y']

    section_name = get_etabs_column_concrete_section(column)

    AddByCoord = {
        'XI':x,
        'YI':y,
        'ZI':column[1]['z_base'],
        'XJ':x,
        'YJ':y,
        'ZJ':column[1]['z_top'],
        'Name':str(name),
        'PropName' : section_name,
        'UserName' : str(name),
        'CSys' : 'Global'
    }
    return AddByCoord

def add_column(SapModel, column, all_columns):
    ret = SapModel.FrameObj.AddByCoord(**get_column_coordinate(column))
    column_name = str(column[0])
    angle = float(column[1]['angle'])
    # assign frame local axis angle
    ret = SapModel.FrameObj.SetLocalAxes(column_name, angle)


_ = [add_column(SapModel, x, sorted_columns) for x in sorted_columns]

# add beams

In [490]:
with open (beam_cross_section_data, 'r') as file:
    sections = json.load(file)

def get_etabs_beam_cross_section(section_data):
    grade:str = section_data[1]['material_grade']
    if grade.upper().startswith('FC'):
        b = round(float(section_data[1]['b']))
        h = round(float(section_data[1]['h']))
        size = f"{b}X{h}"
        name = f"B {size}-{grade}"
    else:
        name = f"B {section_data[1]['name'][4:]}-{grade}"
    return name


def add_beam_rebar(section_name:str, width_T2:float,depth_T3:float):

    SetRebarBeam = {
        'Name': section_name,
        'MatPropLong':'A615Gr60',
        'MatPropConfine':'A615Gr60',
        'CoverTop':60.0,
        'CoverBot':60.0,
        'TopLeftArea':400.0,
        'TopRightArea':400.0,
        'BotLeftArea':400.0,
        'BotRightArea':400.0
    }
    return SetRebarBeam


def add_beam_sections(SapModel, section_data, unit = N_mm_C):
    ret = SapModel.SetPresentUnits(unit)
    name:str = get_etabs_beam_cross_section(section_data)
    material:str = section_data[1]['material_grade']

    width_T2 = float(section_data[1]['b'])
    depth_T3 = float(section_data[1]['h'])
    #define rectangular frame section property

    ret = SapModel.PropFrame.SetRectangle(name, material, depth_T3, width_T2)
    if material.upper().startswith('FC'):
        ret = SapModel.PropFrame.SetRebarBeam(**add_beam_rebar(name,width_T2, depth_T3))

    if material.upper().startswith('FC'):
        #define frame section property modifiers
        ModValue = [1.0, 1.0, 1.0, 0.1, 0.35, 0.35, 1.0, 1.0]
        ret = SapModel.PropFrame.SetModifiers(name, ModValue)
    return name

etabs_section_names = [add_beam_sections(SapModel,x) for x in sections]

etabs_section_names

['B 300X600-FC30',
 'B 400X600-FC30',
 'B 600X600-FC30',
 'B 500X600-FC30',
 'B 600X800-FC30',
 'B SFA_C203x89-S355']

In [491]:
with open (beam_data, 'r') as file:
    beams = json.load(file)


def get_beam_coordinates(beam):
    name = beam[0]

    x = beam[1]['x']
    y = beam[1]['y']
    z = beam[1]['z']

    section_name = get_etabs_beam_cross_section(beam)

    for indx, x_coor in enumerate(x):
        if indx == len(x) - 1:
            continue
        y_coor = y[indx]
        AddByCoord = {
            'XI':x_coor,
            'YI':y_coor,
            'ZI':z,
            'XJ':x[indx+1],
            'YJ':y[indx+1],
            'ZJ':z,
            'Name':str(name),
            'PropName' : section_name,
            'UserName' : str(name),
            'CSys' : 'Global'
        }
        yield AddByCoord

def add_beam(SapModel, beam):
    for beam_coordinate in get_beam_coordinates(beam):
        ret = SapModel.FrameObj.AddByCoord(**beam_coordinate)

    # assign frame local axis angle
    angle = float(beam[1]['angle'])
    if round(angle,3) != 0:
        name = str(beam[0])
        ret = SapModel.FrameObj.SetLocalAxes(name, angle)


_ = [add_beam(SapModel, x) for x in beams]

# add walls

In [492]:
with open (wall_sections_data, 'r') as file:
    sections = json.load(file)

def get_etabs_wall_cross_section(section_data):
    grade:str = section_data[1]['material_grade']
    if grade.upper().startswith('FC'):
        thk = round(float(section_data[1]['thk']))
        name = f"W {thk}-{grade}"
    else:
        name = f"W {section_data[1]['name'][8:]}-{grade}"
    return name


def add_wall_sections(SapModel, section_data, unit = N_mm_C):
    ret = SapModel.SetPresentUnits(unit)
    name:str = get_etabs_wall_cross_section(section_data)
    material:str = section_data[1]['material_grade']
    thk = round(float(section_data[1]['thk']))

    SetWall = {
        'Name' : name,
        'WallPropType':  etabs.eWallPropType.Specified,
        'ShellType':  etabs.eShellType.ShellThin,
        'MatProp':  material,
        'Thickness':  thk,
        'color': -1,
        'notes' : "",
        'GUID':  ""
    }

    ret = SapModel.PropArea.SetWall(** SetWall)

    if material.upper().startswith('FC'):
        #define frame section property modifiers
        ModValue = [0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 1.0, 1.0, 1.0, 1.0]
        ret = SapModel.PropArea.SetModifiers(name, ModValue)

    return name


etabs_section_names = [add_wall_sections(SapModel,x) for x in sections]
etabs_section_names

['W 300-FC30', 'W 250-FC40']

In [493]:
with open (wall_data, 'r') as file:
    walls = json.load(file)


def get_wall_coordinates(wall):
    name = wall[0]

    x = wall[1]['x']
    y = wall[1]['y']
    z_base = wall[1]['z_base']
    z_top = wall[1]['z_top']

    section_name = get_etabs_wall_cross_section(wall)
    pnt_count = 4
    for indx, x_coor in enumerate(x):
        if indx == len(x) - 1:
            continue

        y_coor = y[indx]

        pnt_x, pnt_y, pnt_z =[],[],[]
        pnt_x = [
            x[indx],
            x[indx+1],
            x[indx+1],
            x[indx]
        ]

        pnt_y = [
            y[indx],
            y[indx+1],
            y[indx+1],
            y[indx]
        ]

        pnt_z = [
            z_base,
            z_base,
            z_top,
            z_top
        ]

        AddByCoord = {
            'NumberPoints':pnt_count,
            'X' : pnt_x,
            'Y' : pnt_y,
            'Z' : pnt_z,
            'Name' : str(name),
            'PropName' : section_name if section_name else 'Default',
            'UserName' : str(name),
            'CSys' : 'Global'
        }
        
        yield AddByCoord


def add_wall(SapModel, wall):
    for wall_coordinate in get_wall_coordinates(wall):
        ret = SapModel.AreaObj.AddByCoord(**wall_coordinate)


_ = [add_wall(SapModel, x) for x in walls]

# add floors

In [494]:
with open (floor_sections_data, 'r') as file:
    sections = json.load(file)

def get_etabs_floor_cross_section(section_data):
    grade:str = section_data[1]['material_grade']
    if grade.upper().startswith('FC'):
        thk = round(float(section_data[1]['thk']))
        name = f"S {thk}-{grade}"
    else:
        name = f"S {section_data[1]['name'][8:]}-{grade}"
    return name


def add_floor_sections(SapModel, section_data, unit = N_mm_C):
    ret = SapModel.SetPresentUnits(unit)
    name:str = get_etabs_floor_cross_section(section_data)
    material:str = section_data[1]['material_grade']
    thk = round(float(section_data[1]['thk']))

    SetSlab = {
        'Name' : name,
        'SlabType':  etabs.eSlabType.Slab,
        'ShellType':  etabs.eShellType.ShellThin,
        'MatProp':  material,
        'Thickness':  thk,
        'color': -1,
        'notes' : "",
        'GUID':  ""
    }

    ret = SapModel.PropArea.SetSlab(** SetSlab)

    if material.upper().startswith('FC'):
        #define frame section property modifiers
        ModValue = [1.0, 1.0, 1.0, 0.25, 0.25, 0.25, 1.0, 1.0, 1.0, 1.0]
        ret = SapModel.PropArea.SetModifiers(name, ModValue)

    return name


etabs_section_names = [add_floor_sections(SapModel,x) for x in sections]
etabs_section_names

['S 250-FC30',
 'S 200-FC30',
 'S 300-FC30',
 'S 500-FC30',
 'S 150-FC30',
 'S 130-FC30']

In [495]:
with open (floor_data, 'r') as file:
    floors = json.load(file)



def get_floor_coordinates(floor):
    name = floor[0][0]

    section_name = get_etabs_floor_cross_section(floor)
    
    x = floor[1]['x']
    y = floor[1]['y']
    z = floor[1]['z']

    floor_slab_count = len(x)
    for i in range(floor_slab_count):
        slab_pnt_count = len(x[i])
        pnt_x = x[i]
        pnt_y = y[i]
        pnt_z = [z] * slab_pnt_count

        if floor_slab_count > 1:
            counted_name = f'{name}_{i}'
        else:
            counted_name = name
        AddByCoord = {
            'NumberPoints':slab_pnt_count-1,
            'X' : pnt_x[:slab_pnt_count-1],
            'Y' : pnt_y[:slab_pnt_count-1],
            'Z' : pnt_z[:slab_pnt_count-1],
            'Name' : str(counted_name),
            'PropName' : section_name if section_name else 'Default',
            'UserName' : str(counted_name),
            'CSys' : 'Global'
        }
        
        yield AddByCoord


def add_floor(SapModel, floor):
    for floor_coordinate in get_floor_coordinates(floor):
        ret = SapModel.AreaObj.AddByCoord(**floor_coordinate)


#refresh view
ret = SapModel.View.RefreshView(0, False)


_ = [add_floor(SapModel, x) for x in floors]

0

'6003577'

0

'6004916'

0

...

'6003570_1'

0

'6003570_2'

0

'6003584'

# finish

In [480]:
SapModel = None
myETABSObject = None